In [2]:
# !wget https://huggingface.co/datasets/mesolitica/malaysian-ultrachat/resolve/main/ultrachat-lom-agc.jsonl

In [3]:
!nvidia-smi

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

Thu Feb 15 10:39:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          On  | 00000001:00:00.0 Off |                    0 |
| N/A   36C    P0              63W / 300W |  53652MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [4]:
from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel('BAAI/bge-m3',  use_fp16=True, device = 'cuda')

Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

loading existing colbert_linear and sparse_linear---------


In [5]:
import json
import openai
import torch
from tqdm import tqdm

In [6]:
contexts, questions = [], []

with open('ultrachat-lom-agc.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        if l[1]['content_ms']:
            c = l[0]['content'].strip()
            q = l[1]['content_ms'].strip()
            if not len(c) or not len(q):
                continue
            contexts.append(c)
            questions.append(q)
            
len(contexts), len(questions)

(8034, 8034)

In [7]:
contexts_v = model.encode(contexts, 
                            batch_size=12, 
                            max_length=8192,
                            )['dense_vecs']

encoding: 100%|██████████| 670/670 [01:45<00:00,  6.37it/s]


In [8]:
questions_v = model.encode(questions, 
                            batch_size=12, 
                            max_length=8192,
                            )['dense_vecs']

encoding: 100%|██████████| 670/670 [00:28<00:00, 23.71it/s]


In [9]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [10]:
tops = {
    1: 0,
    3: 0,
    5: 0,
    10: 0,
}

for i in tqdm(range(len(questions_v))):
    argsort = np.argsort(cosine_similarity(questions_v[i].reshape(1, -1), contexts_v)[0])[::-1]
    for k in tops.keys():
        if i in argsort[:k]:
            tops[k] += 1

100%|██████████| 8034/8034 [05:24<00:00, 24.76it/s]


In [12]:
for k, v in tops.items():
    print(k, v / len(questions_v))

1 0.35001244709982576
3 0.5311177495643515
5 0.6011949215832711
10 0.6928055763007219
